<a href="https://colab.research.google.com/github/vparro/MLprobabilistico/blob/master/KLB_estradas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas**

In [ ]:
%%capture
# instala biblioteca para leitura de excel e biblioteca gráfica
!pip install xlrd
!pip install -U plotly
# !pip install plotly==4.8.1

# Exportando figuras com melhores resoluções
!pip install -U kaleido

In [ ]:
%%capture
# Importa a biblioteca pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.plotting.backend = "plotly"

# Importa biblioteca para conversão de datas
from datetime import datetime

# Bilbioteca gráfica utilizada neste estudo
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# biblioteca numérica
import scipy.stats as stats
import scipy
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [ ]:
# Diretório de imagens
import os
if not os.path.exists("images"):
    os.mkdir("images")

# **Funções proprietárias**

In [ ]:
def graphics_plot_prf(y, y_name, yaxes_title, title='Title', bins=1, x_tick=1, save=False, save_name=None):

  '''
  y = [y1, y2, y3, y4]
  y_name = ['y1 name', 'y2 name', 'y3 name', 'y4 name']
  bins = number of bins for x scale
  title = 'Title'
  yaxes_title = ['Title yaxes1', 'Title yaxes2', 'Title yaxes3', 'Title yaxes4']
  x_tick = dtick for axis x
  save = True -> save graphics as image (images/title.eps) 
  '''

  # Create figure with secondary y-axis
  fig = make_subplots(specs=[[{'secondary_y': True}]])

  # Markers
#   markers = ['circle','star','diamond','x']

  # Lines
  dash = ['dash','dot','dashdot']

  for i, v in enumerate(y):
    if (len(yaxes_title) == 1) | (i < len(y) - 1):
      on = False
    else:
      on = True

    if i == 0:
      # Add traces
      fig.add_trace(
          go.Scatter(x=np.linspace(0, 24, bins),
          y=v, name=y_name[i],
        #   marker_symbol=markers[i],
        #   marker_size=7,
          mode='lines',
          line=dict(color='rgb(0, 0, 0)',
          width=2)),
          # line_shape='spline'),
          secondary_y=False)
      
    else:
      # Add traces
      fig.add_trace(
          go.Scatter(x=np.linspace(0, 24, bins),
          y=v, name=y_name[i],
        #   marker_symbol=markers[i],
        #   marker_size=7,
          mode='lines',
          line=dict(color='rgb(0, 0, 0)',
          width=2, 
          dash=dash[i-1])),
          # line_shape='spline'),
          secondary_y=on)
  
  # Add figure title
  fig.update_layout(title_text=title)
  
  # Set x-axis title
  fig.update_xaxes(title_text='Time of day')

  # Set y-axes titles
  if len(yaxes_title) > 1:
    fig.update_yaxes(title_text='<b>'+ yaxes_title[0] +'</b>', secondary_y=False)
    fig.update_yaxes(title_text='<b>'+ yaxes_title[1] +'</b>', secondary_y=True)
  else:
    fig.update_yaxes(title_text='<b>'+ yaxes_title[0] +'</b>', secondary_y=False)

  fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='black')
  fig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='black')

  fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
  fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

  fig.update_layout(paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)')

  fig.update_layout(xaxis=dict(tickmode='linear',
                               tick0=0,
                               dtick=x_tick))

  fig.update_layout(legend=dict(orientation='h',
                                yanchor='bottom',
                                y=1.01,
                                xanchor='right',
                                x=0.95))
  fig.show()

  if save:
    fig.write_image('images/'+ save_name +'.png')

# **Dados Fluxo**

In [ ]:
%%capture
!gdown 1Kru8e3M6AjhEDuVXH7Tqx36wZjz4-Xc0

In [ ]:
# Fluxo
dfFluxo = pd.read_csv('DNIT.csv') 

# dfFluxo['avgtotal'].loc[2::3]   ## [start:stop:step] - se ficar livre vai até o final
dfDNIT = pd.DataFrame(dfFluxo['avgtotal'].loc[2::3].values.reshape(4,24)).T
dfDNIT.tail(3)

,0,1,2,3
21,217.674901,172.611702,164.695324,160.408976
22,183.319164,143.476845,136.172760,132.808623
23,131.100690,104.440947,97.717366,94.281760


# **Estradas federais - Geral**

**2013**

In [ ]:
%%capture
!gdown 13TZAmFfbKhEYuIbYIMOW1NMISGXP0JyO

In [ ]:
df13 = pd.read_csv('datatran2013.csv', encoding = "ISO-8859-1", sep=';')
df13['Hora'] = pd.to_datetime(df13['horario'], format='%H:%M:%S').dt.hour
df13.shape

(186748, 27)

In [ ]:
df13.head(2)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,ano,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,Hora
0,1228858,2013-01-01,Terça,01:00:00,PA,163,992.0,SANTAREM,Defeito mecânico em veículo,Queda de motocicleta / bicicleta / veículo,...,2013,2,0,2,0,0,0,2,1,1
1,1228860,2013-01-01,Terça,01:00:00,PR,476,114.0,COLOMBO,Velocidade incompatível,Colisão com objeto fixo,...,2013,1,0,0,0,1,0,0,1,1


In [ ]:
counts, bins = np.histogram(df13['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='2013 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2013')

**2014**

In [ ]:
%%capture
!gdown 13VRBoeuJpEOc-5iFBovLvZCP8RFFVaml

In [ ]:
df14 = pd.read_csv('datatran2014.csv', encoding = "ISO-8859-1", sep=';')
df14['Hora'] = pd.to_datetime(df14['horario'], format='%H:%M:%S').dt.hour
df14.shape

(169201, 27)

In [ ]:
df14.head(2)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,ano,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,Hora
0,1175,2014-11-22,Sábado,19:20:00,BA,116,778.1,PLANALTO,Ingestão de álcool,Atropelamento de pessoa,...,2014,2,0,0,1,1,0,1,1,19
1,12,2014-11-22,Sábado,08:30:00,SP,116,212.0,GUARULHOS,Outras,Colisão com objeto fixo,...,2014,1,0,0,0,0,1,0,1,8


In [ ]:
counts, bins = np.histogram(df14['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['2014 - Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2014')

**2015**

In [ ]:
%%capture
!gdown 13f3pn98iHub-p5hN1Hzho49Xo9UHp96N

In [ ]:
df15 = pd.read_csv('datatran2015.csv', encoding = "ISO-8859-1", sep=';')
df15['Hora'] = pd.to_datetime(df15['horario'], format='%H:%M:%S').dt.hour
df15.shape

(122161, 27)

In [ ]:
df15.head(2)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,ano,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,Hora
0,83298209,2015-01-01,Quinta,00:05:00,RJ,101,327.0,NITEROI,Outras,Colisão traseira,...,2015,2,0,0,0,2,0,0,2,0
1,83298218,2015-01-01,Quinta,01:20:00,MS,163,434.4,CAMPO GRANDE,Animais na Pista,Saída de Pista,...,2015,2,0,1,0,1,0,1,1,1


In [ ]:
counts, bins = np.histogram(df15['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='2015 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2015')

**2016**

In [ ]:
%%capture
!gdown 13Zc1iIdYWnQ2oQSn8o6CcxZHwrtn6jcu

In [ ]:
df16 = pd.read_csv('datatran2016.csv', encoding = "ISO-8859-1", sep=';')
df16['Hora'] = pd.to_datetime(df16['horario'], format='%H:%M:%S').dt.hour
df16.shape

(96363, 26)

In [ ]:
df16.head(2)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,Hora
0,36727,10/06/16,Sexta,18:30:00,RJ,101,66,CAMPOS DOS GOYTACAZES,Outras,Colisão com objeto fixo,...,Urbano,2,0,0,0,2,0,0,2,18
1,83425846,01/01/16,Sexta,01:30:00,SC,101,"135,5",BALNEARIO CAMBORIU,Falta de atenção,Colisão traseira,...,Urbano,2,0,1,0,1,0,1,2,1


In [ ]:
counts, bins = np.histogram(df16['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='2016 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2016')

**2017**

In [ ]:
%%capture
!gdown 143SYBlyOq2CMO1ySbIvDFkMn9PW8Vi7d

In [ ]:
df17 = pd.read_csv('datatran2017.csv', encoding = "ISO-8859-1", sep=';')
df17['Hora'] = pd.to_datetime(df17['horario'], format='%H:%M:%S').dt.hour
df17.shape

(89567, 31)

In [ ]:
df17.head(2)

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop,Hora
0,17,2017-01-01,domingo,01:45:00,RS,116.0,"34,9",VACARIA,Defeito Mecânico no Veículo,Colisão traseira,...,2,0,4,2,"-28,5071196","-50,941176",SPRF-RS,DEL05-RS,UOP03-DEL05-RS,1
1,20,2017-01-01,domingo,01:00:00,PR,376.0,636,TIJUCAS DO SUL,Velocidade Incompatível,Saída de leito carroçável,...,1,0,0,2,"-25,754","-49,1266",SPRF-PR,DEL01-PR,DEL7/1-UOP08/PR,1


In [ ]:
counts, bins = np.histogram(df17['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='2017 - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='geral_2017')

## **R² e p-value**

In [ ]:
# Y = stats.zscore(ys)
# X_m = stats.zscore(sinusoid(X, *paramy))
# X_m = sm.add_constant(X_m)
# model = sm.OLS(Y,X_m)
# results = model.fit()
# results.params

# y5 = results.predict(X_m)
# print(results.summary())

# **Dados já filtrados para um único veículo**

In [ ]:
%%capture 
!gdown 1rLrFckRbr_oScvziSCodjC2_UxoEOAMk # PRF 2013 - 1 veículo
!gdown 1--JAT8gfd9CBtIlNBBj5yB_C5ipq9p0u # PRF 2014 - 1 veículo
!gdown 1-4vkJOgGW9W__P9zFhbVQkT3Uarwmbbv # PRF 2015 - 1 veículo
!gdown 1-6cj1H9JsUtdIZtO78x3HhdSFjpsqPj_ # PRF 2016 - 1 veículo
!gdown 1-6tc2J-omr-39k9n3FWApTqnJrZFIbnp # PRF 2017 - 1 veículo

In [ ]:
# Datasets já filtrados para veículo único
prf13 = pd.read_csv('prf_2013_1veiculo.csv')
prf13['Hora'] = pd.to_datetime(prf13['horario'], format='%H:%M:%S').dt.hour
prf14 = pd.read_csv('prf_2014_1veiculo.csv')
prf14['Hora'] = pd.to_datetime(prf14['horario'], format='%H:%M:%S').dt.hour
prf15 = pd.read_csv('prf_2015_1veiculo.csv')
prf15['Hora'] = pd.to_datetime(prf15['horario'], format='%H:%M:%S').dt.hour
prf16 = pd.read_csv('prf_2016_1veiculo.csv')
prf16['Hora'] = pd.to_datetime(prf16['horario'], format='%H:%M:%S').dt.hour
prf17 = pd.read_csv('prf_2017_1veiculo.csv')
prf17['Hora'] = pd.to_datetime(prf17['horario'], format='%H:%M:%S').dt.hour

# **Estrada simples + dupla**

In [ ]:
# 2013
prf13.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla'], dtype=object)

In [ ]:
# 2013 somente com pista "Simples" e "Dupla"
df13_simples_dupla = prf13[prf13.tipo_pista.isin(['Simples', 'Dupla'])]
df13_simples_dupla.tipo_pista.unique()

array(['Simples', 'Dupla'], dtype=object)

In [ ]:
# 2014
prf14.tipo_pista.unique()

array(['Simples', 'Múltipla', 'Dupla'], dtype=object)

In [ ]:
# 2014 somente com pista "Simples" e "Dupla"
df14_simples_dupla = prf14[prf14.tipo_pista.isin(['Simples', 'Dupla'])]
df14_simples_dupla.tipo_pista.unique()

array(['Simples', 'Dupla'], dtype=object)

In [ ]:
# 2015
prf15.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla', '(null)'], dtype=object)

In [ ]:
# 2015 somente com pista "Simples" e "Dupla"
df15_simples_dupla = prf15[prf15.tipo_pista.isin(['Simples', 'Dupla'])]
df15_simples_dupla.tipo_pista.unique()

array(['Simples', 'Dupla'], dtype=object)

In [ ]:
# 2016
prf16.tipo_pista.unique()

array(['Dupla', 'Simples', 'Múltipla'], dtype=object)

In [ ]:
# 2016 somente com pista "Simples" e "Dupla"
df16_simples_dupla = prf16[prf16.tipo_pista.isin(['Simples', 'Dupla'])]
df16_simples_dupla.tipo_pista.unique()

array(['Dupla', 'Simples'], dtype=object)

In [ ]:
# 2017
prf17.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla'], dtype=object)

In [ ]:
# 2017 somente com pista "Simples" e "Dupla"
df17_simples_dupla = prf17[prf17.tipo_pista.isin(['Simples', 'Dupla'])]
df17_simples_dupla.tipo_pista.unique()

array(['Simples', 'Dupla'], dtype=object)

## **Um veículo + Reta + Caminhões + Capotamento ou Tombamento**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples_dupla.veiculos.unique()

array([1])

In [ ]:
df13_simples_dupla.tipo_veiculo.unique()

array(['Motocicletas', 'Automóvel', 'Caminhão', 'Não identificado',
       'Ônibus', '(null)', 'Utilitário', 'Caminhonete', 'Bicicleta',
       'Caminhão-Trator', 'Motoneta', 'Camioneta', 'Microônibus',
       'Triciclo', 'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque',
       'Carroça', 'Semi-Reboque', 'Reboque', 'Bonde / Trem',
       'Trator misto'], dtype=object)

In [ ]:
df13_simples_dupla.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo',
       'Colisão com objeto fixo', 'Saída de Pista',
       'Atropelamento de pessoa', 'Capotamento',
       'Atropelamento de animal', 'Tombamento', 'Derramamento de Carga',
       'Incêndio', 'Danos Eventuais', 'Colisão com objeto móvel',
       'Colisão traseira', 'Colisão lateral', 'Colisão Transversal',
       'Colisão frontal'], dtype=object)

In [ ]:
# Filtragem 'Reta' + Caminhões + 'Tombamento' ou 'Capotamento'
reta = df13_simples_dupla.tracado_via == 'Reta'

list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df13_simples_dupla.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df13_simples_dupla.tipo_acidente.isin(list_acid)

df13_sd_reta_capot_tomb = df13_simples_dupla[(reta) & (caminhao) & (capot_tomb)]

In [ ]:
df13_sd_reta_capot_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df13_sd_reta_capot_tomb.tipo_veiculo.unique()

array(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df13_sd_reta_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_sd_reta_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='Trucks on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples_dupla.veiculos.unique()

array([1])

In [ ]:
df14_simples_dupla.tipo_veiculo.unique()

array(['Caminhonete', 'Automóvel', 'Caminhão-Trator', '(null)',
       'Motocicletas', 'Motoneta', 'Caminhão', 'Camioneta', 'Ciclomotor',
       'Ônibus', 'Utilitário', 'Microônibus', 'Não identificado',
       'Semi-Reboque', 'Bicicleta', 'Reboque', 'Caminhão-Tanque',
       'Trator misto', 'Trator de rodas', 'Triciclo',
       'Trator de esteiras'], dtype=object)

In [ ]:
df14_simples_dupla.tipo_acidente.unique()

array(['Atropelamento de pessoa', 'Saída de Pista',
       'Colisão com objeto fixo', 'Atropelamento de animal', 'Incêndio',
       'Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Tombamento', 'Derramamento de Carga', 'Danos Eventuais',
       'Colisão com objeto móvel', 'Colisão com bicicleta',
       'Colisão traseira', 'Colisão Transversal', 'Colisão lateral',
       'Colisão frontal'], dtype=object)

In [ ]:
# Filtragem 'Reta' + Caminhões + 'Tombamento' ou 'Capotamento'
reta = df14_simples_dupla.tracado_via == 'Reta'

list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df14_simples_dupla.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df14_simples_dupla.tipo_acidente.isin(list_acid)

df14_sd_reta_capot_tomb = df14_simples_dupla[(reta) & (caminhao) & (capot_tomb)]

In [ ]:
df14_sd_reta_capot_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df14_sd_reta_capot_tomb.tipo_veiculo.unique()

array(['Caminhão-Trator', 'Caminhão', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df14_sd_reta_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_sd_reta_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='Trucks on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples_dupla.veiculos.unique()

array([1])

In [ ]:
df15_simples_dupla.tipo_veiculo.unique()

array(['Automóvel', 'Caminhão', 'Motocicletas', 'Caminhão-Trator',
       '(null)', 'Camioneta', 'Motoneta', 'Utilitário', 'Caminhonete',
       'Microônibus', 'Não identificado', 'Ônibus', 'Bicicleta',
       'Ciclomotor', 'Trator de rodas', 'Caminhão-Tanque', 'Triciclo',
       'Trator de esteiras', 'Semi-Reboque', 'Reboque', 'Trator misto',
       'Quadriciclo', 'Carroça', 'Motor-Casa'], dtype=object)

In [ ]:
df15_simples_dupla.tipo_acidente.unique()

array(['Saída de Pista', 'Colisão com objeto fixo', 'Tombamento',
       'Queda de motocicleta / bicicleta / veículo',
       'Atropelamento de animal', 'Capotamento',
       'Atropelamento de pessoa', 'Incêndio', 'Danos Eventuais',
       'Colisão com objeto móvel', 'Derramamento de Carga'], dtype=object)

In [ ]:
# Filtragem 'Reta' + Caminhões + 'Tombamento' ou 'Capotamento'
reta = df15_simples_dupla.tracado_via == 'Reta'

list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df15_simples_dupla.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df15_simples_dupla.tipo_acidente.isin(list_acid)

df15_sd_reta_capot_tomb = df15_simples_dupla[(reta) & (caminhao) & (capot_tomb)]

In [ ]:
df15_sd_reta_capot_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_sd_reta_capot_tomb.tipo_veiculo.unique()

array(['Caminhão-Trator', 'Caminhão', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df15_sd_reta_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_sd_reta_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='Trucks on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples_dupla.veiculos.unique()

array([1])

In [ ]:
df16_simples_dupla.tipo_veiculo.unique()

array(['Motocicletas', 'Camioneta', 'Automóvel', 'Caminhão-Trator',
       'Caminhonete', 'Microônibus', 'Caminhão', 'Ônibus', 'Utilitário',
       'Motoneta', nan, 'Ciclomotor', 'Não identificado', 'Bicicleta',
       'Caminhão-Tanque', 'Semi-Reboque', 'Motor-Casa', 'Triciclo',
       'Trator de rodas', 'Carroça', 'Reboque', 'Trator de esteiras',
       'Carro-de-mao', 'Charrete'], dtype=object)

In [ ]:
df16_simples_dupla.tipo_acidente.unique()

array(['Queda de motocicleta / bicicleta / veículo', 'Capotamento',
       'Saída de Pista', 'Colisão com objeto fixo', 'Incêndio',
       'Tombamento', 'Atropelamento de pessoa',
       'Colisão com objeto móvel', 'Atropelamento de animal',
       'Derramamento de Carga', 'Danos Eventuais'], dtype=object)

In [ ]:
# Filtragem 'Reta' + Caminhões + 'Tombamento' ou 'Capotamento'
reta = df16_simples_dupla.tracado_via == 'Reta'

list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
caminhao = df16_simples_dupla.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df16_simples_dupla.tipo_acidente.isin(list_acid)

df16_sd_reta_capot_tomb = df16_simples_dupla[(reta) & (caminhao) & (capot_tomb)]

In [ ]:
df16_sd_reta_capot_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_sd_reta_capot_tomb.tipo_veiculo.unique()

array(['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque'], dtype=object)

In [ ]:
df16_sd_reta_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_sd_reta_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='Trucks on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df17_simples_dupla.veiculos.unique()

array([1])

In [ ]:
df17_simples_dupla.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão', 'Semireboque', 'Reboque',
       'Motocicleta', 'Automóvel', 'Caminhonete', 'Bicicleta',
       'Camioneta', 'Motoneta', 'Micro-ônibus', 'Utilitário', 'Outros',
       'Ônibus', 'Não Informado', 'Trator de rodas', 'Ciclomotor',
       'Trator de esteira', 'Triciclo', 'Trator misto'], dtype=object)

In [ ]:
df17_simples_dupla.tipo_acidente.unique()

array(['Colisão com objeto estático', 'Tombamento',
       'Derramamento de carga', 'Atropelamento de Pedestre',
       'Saída de leito carroçável', 'Incêndio', 'Atropelamento de Animal',
       'Capotamento', 'Colisão com objeto em movimento',
       'Danos eventuais', 'Colisão lateral', 'Colisão traseira',
       'Queda de ocupante de veículo', 'Colisão frontal', 'Engavetamento',
       'Colisão transversal'], dtype=object)

In [ ]:
# Filtragem 'Reta' + Caminhões + 'Tombamento' ou 'Capotamento'
reta = df17_simples_dupla.tracado_via == 'Reta'

list_caminhao = ['Caminhão', 'Caminhão-trator']
caminhao = df17_simples_dupla.tipo_veiculo.isin(list_caminhao)

list_acid = ['Capotamento', 'Tombamento']
capot_tomb = df17_simples_dupla.tipo_acidente.isin(list_acid)

df17_sd_reta_capot_tomb = df17_simples_dupla[(reta) & (caminhao) & (capot_tomb)]

In [ ]:
df17_sd_reta_capot_tomb.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_sd_reta_capot_tomb.tipo_veiculo.unique()

array(['Caminhão-trator', 'Caminhão'], dtype=object)

In [ ]:
df17_sd_reta_capot_tomb.tipo_acidente.unique()

array(['Tombamento', 'Capotamento'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_sd_reta_capot_tomb['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='Trucks on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_2017')

## **Um veículo + Reta + feridos**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 32, 34, 37, 39, 40, 41, 42,
       46, 49])

In [ ]:
# df13_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df13_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df13_simples_dupla.tracado_via == 'Reta'

feridos = df13_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples_dupla.tipo_acidente.isin(list_acid)

df13_sd_reta_feridos = df13_simples_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13_sd_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df13_sd_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18,
       20, 21, 22, 23, 25, 27, 28, 32, 37, 40, 42])

In [ ]:
# df13_sd_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df13_sd_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17,
       19, 21, 22, 23, 24, 25, 26, 27, 29, 34, 36, 37, 38, 43, 45, 47, 49])

In [ ]:
# df14_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df14_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df14_simples_dupla.tracado_via == 'Reta'

feridos = df14_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples_dupla.tipo_acidente.isin(list_acid)

df14_sd_reta_feridos = df14_simples_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14_sd_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df14_sd_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 14, 15, 16, 17, 19, 21,
       22, 23, 25, 27, 29, 34, 43, 45, 47, 49])

In [ ]:
# df14_sd_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df14_sd_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 31, 33, 35, 37, 40, 41, 44, 46])

In [ ]:
# df15_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df15_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df15_simples_dupla.tracado_via == 'Reta'

feridos = df15_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples_dupla.tipo_acidente.isin(list_acid)

df15_sd_reta_feridos = df15_simples_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15_sd_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df15_sd_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 15, 18, 20, 31, 37,
       44])

In [ ]:
# df15_sd_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df15_sd_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 20,
       21, 23, 24, 25, 27, 28, 29, 31, 34, 41, 47, 50])

In [ ]:
# df16_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df16_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df16_simples_dupla.tracado_via == 'Reta'

feridos = df16_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples_dupla.tipo_acidente.isin(list_acid)

df16_sd_reta_feridos = df16_simples_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16_sd_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df16_sd_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 20, 21, 23,
       25, 28, 29, 31, 34, 41, 50])

In [ ]:
# df16_sd_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df16_sd_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df17_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 24, 25, 26, 27, 28, 31, 34, 35, 37, 45, 62])

In [ ]:
# df17_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df17_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df17_simples_dupla.tracado_via == 'Reta'

feridos = df17_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples_dupla.tipo_acidente.isin(list_acid)

df17_sd_reta_feridos = df17_simples_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17_sd_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df17_sd_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 25, 28, 35, 37, 62])

In [ ]:
# df17_sd_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df17_sd_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_feridos_2017')

## **Um veículo + Curva**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df13_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df13_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df13_simples_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples_dupla.tipo_acidente.isin(list_acid)

df13_sd_curva = df13_simples_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13_sd_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df13_sd_curva.tipo_veiculo.unique()

In [ ]:
# df13_sd_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df14_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df14_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df14_simples_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples_dupla.tipo_acidente.isin(list_acid)

df14_sd_curva = df14_simples_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14_sd_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df14_sd_curva.tipo_veiculo.unique()

In [ ]:
# df14_sd_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df15_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df15_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df15_simples_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples_dupla.tipo_acidente.isin(list_acid)

df15_sd_curva = df15_simples_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15_sd_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df15_sd_curva.tipo_veiculo.unique()

In [ ]:
# df15_sd_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df16_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df16_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df16_simples_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples_dupla.tipo_acidente.isin(list_acid)

df16_sd_curva = df16_simples_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16_sd_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df16_sd_curva.tipo_veiculo.unique()

In [ ]:
# df16_sd_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df17_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df17_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df17_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df17_simples_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples_dupla.tipo_acidente.isin(list_acid)

df17_sd_curva = df17_simples_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17_sd_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df17_sd_curva.tipo_veiculo.unique()

In [ ]:
# df17_sd_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_2017')

## **Um veículo + Curva + feridos**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 32, 34, 37, 39, 40, 41, 42,
       46, 49])

In [ ]:
# df13_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df13_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df13_simples_dupla.tracado_via == 'Curva'

feridos = df13_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples_dupla.tipo_acidente.isin(list_acid)

df13_sd_curva_feridos = df13_simples_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13_sd_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df13_sd_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 19,
       23, 25, 34, 39, 41, 46, 49])

In [ ]:
# df13_sd_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df13_sd_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17,
       19, 21, 22, 23, 24, 25, 26, 27, 29, 34, 36, 37, 38, 43, 45, 47, 49])

In [ ]:
# df14_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df14_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df14_simples_dupla.tracado_via == 'Curva'

feridos = df14_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples_dupla.tipo_acidente.isin(list_acid)

df14_sd_curva_feridos = df14_simples_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14_sd_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df14_sd_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 24, 26, 36,
       37])

In [ ]:
# df14_sd_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df14_sd_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 31, 33, 35, 37, 40, 41, 44, 46])

In [ ]:
# df15_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df15_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df15_simples_dupla.tracado_via == 'Curva'

feridos = df15_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples_dupla.tipo_acidente.isin(list_acid)

df15_sd_curva_feridos = df15_simples_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15_sd_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df15_sd_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 20,
       21, 22, 31, 33, 35, 40, 41, 44, 46])

In [ ]:
# df15_sd_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df15_sd_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 20,
       21, 23, 24, 25, 27, 28, 29, 31, 34, 41, 47, 50])

In [ ]:
# df16_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df16_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df16_simples_dupla.tracado_via == 'Curva'

feridos = df16_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples_dupla.tipo_acidente.isin(list_acid)

df16_sd_curva_feridos = df16_simples_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16_sd_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df16_sd_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 13, 18, 24, 27, 47])

In [ ]:
# df16_sd_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df16_sd_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df17_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 24, 25, 26, 27, 28, 31, 34, 35, 37, 45, 62])

In [ ]:
# df17_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df17_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df17_simples_dupla.tracado_via == 'Curva'

feridos = df17_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples_dupla.tipo_acidente.isin(list_acid)

df17_sd_curva_feridos = df17_simples_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17_sd_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df17_sd_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
       24, 27, 34, 45])

In [ ]:
# df17_sd_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df17_sd_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_curva_feridos_2017')

## **Um veículo + Reta + Curva**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df13_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df13_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df13_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples_dupla.tipo_acidente.isin(list_acid)

df13_sd_reta_curva = df13_simples_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13_sd_reta_curva.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
# df13_sd_reta_curva.tipo_veiculo.unique()

In [ ]:
# df13_sd_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df14_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df14_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df14_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples_dupla.tipo_acidente.isin(list_acid)

df14_sd_reta_curva = df14_simples_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14_sd_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df14_sd_reta_curva.tipo_veiculo.unique()

In [ ]:
# df14_sd_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df15_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df15_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df15_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples_dupla.tipo_acidente.isin(list_acid)

df15_sd_reta_curva = df15_simples_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15_sd_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df15_sd_reta_curva.tipo_veiculo.unique()

In [ ]:
# df15_sd_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df16_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df16_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df16_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples_dupla.tipo_acidente.isin(list_acid)

df16_sd_reta_curva = df16_simples_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16_sd_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df16_sd_reta_curva.tipo_veiculo.unique()

In [ ]:
# df16_sd_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df17_simples_dupla.veiculos.unique()

array([1])

In [ ]:
# df17_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df17_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df17_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples_dupla.tipo_acidente.isin(list_acid)

df17_sd_reta_curva = df17_simples_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17_sd_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df17_sd_reta_curva.tipo_veiculo.unique()

In [ ]:
# df17_sd_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_2017')

## **Um veículo + Reta + Curva + feridos**

**2013**

In [ ]:
df13_simples_dupla.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 32, 34, 37, 39, 40, 41, 42,
       46, 49])

In [ ]:
# df13_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df13_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df13_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df13_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples_dupla.tipo_acidente.isin(list_acid)

df13_sd_reta_curva_feridos = df13_simples_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13_sd_reta_curva_feridos.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
np.sort(df13_sd_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 25, 27, 28, 32, 34, 37, 39, 40, 41, 42, 46,
       49])

In [ ]:
# df13_sd_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df13_sd_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2013')

**2014**

In [ ]:
df14_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17,
       19, 21, 22, 23, 24, 25, 26, 27, 29, 34, 36, 37, 38, 43, 45, 47, 49])

In [ ]:
# df14_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df14_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df14_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df14_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples_dupla.tipo_acidente.isin(list_acid)

df14_sd_reta_curva_feridos = df14_simples_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14_sd_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df14_sd_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 19,
       21, 22, 23, 24, 25, 26, 27, 29, 34, 36, 37, 43, 45, 47, 49])

In [ ]:
# df14_sd_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df14_sd_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2014')

**2015**

In [ ]:
df15_simples_dupla.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 31, 33, 35, 37, 40, 41, 44, 46])

In [ ]:
# df15_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df15_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df15_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df15_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples_dupla.tipo_acidente.isin(list_acid)

df15_sd_reta_curva_feridos = df15_simples_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15_sd_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df15_sd_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 20, 21, 22, 31, 33, 35, 37, 40, 41, 44, 46])

In [ ]:
# df15_sd_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df15_sd_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2015')

**2016**

In [ ]:
df16_simples_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 20,
       21, 23, 24, 25, 27, 28, 29, 31, 34, 41, 47, 50])

In [ ]:
# df16_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df16_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df16_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df16_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples_dupla.tipo_acidente.isin(list_acid)

df16_sd_reta_curva_feridos = df16_simples_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16_sd_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df16_sd_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 20, 21,
       23, 24, 25, 27, 28, 29, 31, 34, 41, 47, 50])

In [ ]:
# df16_sd_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df16_sd_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2016')

**2017**

In [ ]:
df17_simples_dupla.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
df17_simples_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_simples_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 24, 25, 26, 27, 28, 31, 34, 35, 37, 45, 62])

In [ ]:
# df17_simples_dupla.tipo_veiculo.unique()

In [ ]:
# df17_simples_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df17_simples_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df17_simples_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_simples_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples_dupla.tipo_acidente.isin(list_acid)

df17_sd_reta_curva_feridos = df17_simples_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17_sd_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df17_sd_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 24, 25, 27, 28, 34, 35, 37, 45, 62])

In [ ]:
# df17_sd_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df17_sd_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17_sd_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_dupla_reta_curva_feridos_2017')

# **Estrada simples**

In [ ]:
# 2013
prf13.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla'], dtype=object)

In [ ]:
# 2013 somente com pista "Simples"
df13_simples = prf13[prf13.tipo_pista.isin(['Simples'])]
df13_simples.tipo_pista.unique()

array(['Simples'], dtype=object)

In [ ]:
# 2014
prf14.tipo_pista.unique()

array(['Simples', 'Múltipla', 'Dupla'], dtype=object)

In [ ]:
# 2014 somente com pista "Simples"
df14_simples = prf14[prf14.tipo_pista.isin(['Simples'])]
df14_simples.tipo_pista.unique()

array(['Simples'], dtype=object)

In [ ]:
# 2015
prf15.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla', '(null)'], dtype=object)

In [ ]:
# 2015 somente com pista "Simples"
df15_simples = prf15[prf15.tipo_pista.isin(['Simples'])]
df15_simples.tipo_pista.unique()

array(['Simples'], dtype=object)

In [ ]:
# 2016
prf16.tipo_pista.unique()

array(['Dupla', 'Simples', 'Múltipla'], dtype=object)

In [ ]:
# 2016 somente com pista "Simples"
df16_simples = prf16[prf16.tipo_pista.isin(['Simples'])]
df16_simples.tipo_pista.unique()

array(['Simples'], dtype=object)

In [ ]:
# 2017
prf17.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla'], dtype=object)

In [ ]:
# 2017 somente com pista "Simples"
df17_simples = prf17[prf17.tipo_pista.isin(['Simples'])]
df17_simples.tipo_pista.unique()

array(['Simples'], dtype=object)

## **Um veículo + Reta**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples.veiculos.unique()

array([1])

In [ ]:
# df13_simples.tipo_veiculo.unique()

In [ ]:
# df13_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df13_simples.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples.tipo_acidente.isin(list_acid)

df13s_reta = df13_simples[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13s_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df13s_reta.tipo_veiculo.unique()

In [ ]:
# df13s_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13s_reta['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples.veiculos.unique()

array([1])

In [ ]:
# df14_simples.tipo_veiculo.unique()

In [ ]:
# df14_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df14_simples.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples.tipo_acidente.isin(list_acid)

df14s_reta = df14_simples[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14s_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df14s_reta.tipo_veiculo.unique()

In [ ]:
# df14s_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14s_reta['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples.veiculos.unique()

array([1])

In [ ]:
# df15_simples.tipo_veiculo.unique()

In [ ]:
# df15_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df15_simples.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples.tipo_acidente.isin(list_acid)

df15s_reta = df15_simples[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15s_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df15s_reta.tipo_veiculo.unique()

In [ ]:
# df15s_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15s_reta['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples.veiculos.unique()

array([1])

In [ ]:
# df16_simples.tipo_veiculo.unique()

In [ ]:
# df16_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df16_simples.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples.tipo_acidente.isin(list_acid)

df16s_reta = df16_simples[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16s_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df16s_reta.tipo_veiculo.unique()

In [ ]:
# df16s_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16s_reta['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel', 'Ponte',
       'Retorno Regulamentado'], dtype=object)

In [ ]:
df17_simples.veiculos.unique()

array([1])

In [ ]:
# df17_simples.tipo_veiculo.unique()

In [ ]:
# df17_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df17_simples.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples.tipo_acidente.isin(list_acid)

df17s_reta = df17_simples[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17s_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df17s_reta.tipo_veiculo.unique()

In [ ]:
# df17s_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17s_reta['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_2017')

## **Um veículo + Reta + feridos**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 22, 23, 25, 27, 32, 34, 37, 39, 42, 46, 49])

In [ ]:
# df13_simples.tipo_veiculo.unique()

In [ ]:
# df13_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df13_simples.tracado_via == 'Reta'

feridos = df13_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples.tipo_acidente.isin(list_acid)

df13s_reta_feridos = df13_simples[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13s_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df13s_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 18, 20, 22, 23,
       25, 27, 32, 37, 42])

In [ ]:
# df13s_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df13s_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17,
       19, 21, 22, 23, 24, 26, 27, 29, 34, 36, 37, 38, 43, 45, 47, 49])

In [ ]:
# df14_simples.tipo_veiculo.unique()

In [ ]:
# df14_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df14_simples.tracado_via == 'Reta'

feridos = df14_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples.tipo_acidente.isin(list_acid)

df14s_reta_feridos = df14_simples[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14s_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df14s_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 14, 15, 16, 17, 19, 21, 22,
       23, 27, 29, 34, 43, 45, 47, 49])

In [ ]:
# df14s_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df14s_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 31, 33, 35, 37, 40, 44, 46])

In [ ]:
# df15_simples.tipo_veiculo.unique()

In [ ]:
# df15_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df15_simples.tracado_via == 'Reta'

feridos = df15_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples.tipo_acidente.isin(list_acid)

df15s_reta_feridos = df15_simples[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15s_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df15s_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 15, 18, 20, 31, 37,
       44])

In [ ]:
# df15s_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df15s_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 24,
       29, 34, 47, 50])

In [ ]:
# df16_simples.tipo_veiculo.unique()

In [ ]:
# df16_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df16_simples.tracado_via == 'Reta'

feridos = df16_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples.tipo_acidente.isin(list_acid)

df16s_reta_feridos = df16_simples[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16s_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df16s_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 29, 34, 50])

In [ ]:
# df16s_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df16s_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel', 'Ponte',
       'Retorno Regulamentado'], dtype=object)

In [ ]:
df17_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 24, 25, 28, 31, 34, 35, 37, 45, 62])

In [ ]:
# df17_simples.tipo_veiculo.unique()

In [ ]:
# df17_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df17_simples.tracado_via == 'Reta'

feridos = df17_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples.tipo_acidente.isin(list_acid)

df17s_reta_feridos = df17_simples[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17s_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df17s_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 12, 13, 14, 15, 17, 18, 25, 35,
       37, 62])

In [ ]:
# df17s_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df17s_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17s_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_feridos_2017')

## **Um veículo + Curva**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples.veiculos.unique()

array([1])

In [ ]:
# df13_simples.tipo_veiculo.unique()

In [ ]:
# df13_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df13_simples.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples.tipo_acidente.isin(list_acid)

df13s_curva = df13_simples[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13s_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df13s_curva.tipo_veiculo.unique()

In [ ]:
# df13s_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13s_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples.veiculos.unique()

array([1])

In [ ]:
# df14_simples.tipo_veiculo.unique()

In [ ]:
# df14_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df14_simples.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples.tipo_acidente.isin(list_acid)

df14s_curva = df14_simples[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14s_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df14s_curva.tipo_veiculo.unique()

In [ ]:
# df14s_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14s_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples.veiculos.unique()

array([1])

In [ ]:
# df15_simples.tipo_veiculo.unique()

In [ ]:
# df15_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df15_simples.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples.tipo_acidente.isin(list_acid)

df15s_curva = df15_simples[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15s_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df15s_curva.tipo_veiculo.unique()

In [ ]:
# df15s_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15s_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples.veiculos.unique()

array([1])

In [ ]:
# df16_simples.tipo_veiculo.unique()

In [ ]:
# df16_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df16_simples.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples.tipo_acidente.isin(list_acid)

df16s_curva = df16_simples[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16s_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df16s_curva.tipo_veiculo.unique()

In [ ]:
# df16s_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16s_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel', 'Ponte',
       'Retorno Regulamentado'], dtype=object)

In [ ]:
df17_simples.veiculos.unique()

array([1])

In [ ]:
# df17_simples.tipo_veiculo.unique()

In [ ]:
# df17_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df17_simples.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples.tipo_acidente.isin(list_acid)

df17s_curva = df17_simples[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17s_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df17s_curva.tipo_veiculo.unique()

In [ ]:
# df17s_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17s_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_2017')

## **Um veículo + Curva + feridos**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 22, 23, 25, 27, 32, 34, 37, 39, 42, 46, 49])

In [ ]:
# df13_simples.tipo_veiculo.unique()

In [ ]:
# df13_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df13_simples.tracado_via == 'Curva'

feridos = df13_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples.tipo_acidente.isin(list_acid)

df13s_curva_feridos = df13_simples[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13s_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df13s_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 19,
       34, 39, 46, 49])

In [ ]:
# df13s_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df13s_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17,
       19, 21, 22, 23, 24, 26, 27, 29, 34, 36, 37, 38, 43, 45, 47, 49])

In [ ]:
# df14_simples.tipo_veiculo.unique()

In [ ]:
# df14_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df14_simples.tracado_via == 'Curva'

feridos = df14_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples.tipo_acidente.isin(list_acid)

df14s_curva_feridos = df14_simples[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14s_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df14s_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 15, 24, 26, 36, 37])

In [ ]:
# df14s_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df14s_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 31, 33, 35, 37, 40, 44, 46])

In [ ]:
# df15_simples.tipo_veiculo.unique()

In [ ]:
# df15_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df15_simples.tracado_via == 'Curva'

feridos = df15_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples.tipo_acidente.isin(list_acid)

df15s_curva_feridos = df15_simples[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15s_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df15s_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 17, 20, 21,
       22, 31, 33, 35, 40, 44, 46])

In [ ]:
# df15s_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df15s_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 24,
       29, 34, 47, 50])

In [ ]:
# df16_simples.tipo_veiculo.unique()

In [ ]:
# df16_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df16_simples.tracado_via == 'Curva'

feridos = df16_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples.tipo_acidente.isin(list_acid)

df16s_curva_feridos = df16_simples[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16s_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df16s_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 13, 18, 24, 47])

In [ ]:
# df16s_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df16s_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel', 'Ponte',
       'Retorno Regulamentado'], dtype=object)

In [ ]:
df17_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 24, 25, 28, 31, 34, 35, 37, 45, 62])

In [ ]:
# df17_simples.tipo_veiculo.unique()

In [ ]:
# df17_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df17_simples.tracado_via == 'Curva'

feridos = df17_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples.tipo_acidente.isin(list_acid)

df17s_curva_feridos = df17_simples[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17s_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df17s_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
       24, 27, 34, 45])

In [ ]:
# df17s_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df17s_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17s_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_curva_feridos_2017')

## **Um veículo + Reta + Curva**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples.veiculos.unique()

array([1])

In [ ]:
# df13_simples.tipo_veiculo.unique()

In [ ]:
# df13_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df13_simples.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples.tipo_acidente.isin(list_acid)

df13s_reta_curva = df13_simples[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13s_reta_curva.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
# df13s_reta_curva.tipo_veiculo.unique()

In [ ]:
# df13s_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples.veiculos.unique()

array([1])

In [ ]:
# df14_simples.tipo_veiculo.unique()

In [ ]:
# df14_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df14_simples.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples.tipo_acidente.isin(list_acid)

df14s_reta_curva = df14_simples[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14s_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df14s_reta_curva.tipo_veiculo.unique()

In [ ]:
# df14s_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples.veiculos.unique()

array([1])

In [ ]:
# df15_simples.tipo_veiculo.unique()

In [ ]:
# df15_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df15_simples.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples.tipo_acidente.isin(list_acid)

df15s_reta_curva = df15_simples[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15s_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df15s_reta_curva.tipo_veiculo.unique()

In [ ]:
# df15s_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples.veiculos.unique()

array([1])

In [ ]:
# df16_simples.tipo_veiculo.unique()

In [ ]:
# df16_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df16_simples.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples.tipo_acidente.isin(list_acid)

df16s_reta_curva = df16_simples[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16s_reta_curva.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
# df16s_reta_curva.tipo_veiculo.unique()

In [ ]:
# df16s_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel', 'Ponte',
       'Retorno Regulamentado'], dtype=object)

In [ ]:
df17_simples.veiculos.unique()

array([1])

In [ ]:
# df17_simples.tipo_veiculo.unique()

In [ ]:
# df17_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df17_simples.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples.tipo_acidente.isin(list_acid)

df17s_reta_curva = df17_simples[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17s_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df17s_reta_curva.tipo_veiculo.unique()

In [ ]:
# df17s_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17s_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_2017')

## **Um veículo + Reta + Curva + feridos**

**2013**

In [ ]:
df13_simples.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
df13_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 22, 23, 25, 27, 32, 34, 37, 39, 42, 46, 49])

In [ ]:
# df13_simples.tipo_veiculo.unique()

In [ ]:
# df13_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df13_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df13_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_simples.tipo_acidente.isin(list_acid)

df13s_reta_curva_feridos = df13_simples[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13s_reta_curva_feridos.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
np.sort(df13s_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 25, 27, 28, 32, 34, 37, 39, 40, 41, 42, 46,
       49])

In [ ]:
# df13s_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df13s_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2013')

**2014**

In [ ]:
df14_simples.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17,
       19, 21, 22, 23, 24, 26, 27, 29, 34, 36, 37, 38, 43, 45, 47, 49])

In [ ]:
# df14_simples.tipo_veiculo.unique()

In [ ]:
# df14_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df14_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df14_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_simples.tipo_acidente.isin(list_acid)

df14s_reta_curva_feridos = df14_simples[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14s_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df14s_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 19,
       21, 22, 23, 24, 26, 27, 29, 34, 36, 37, 43, 45, 47, 49])

In [ ]:
# df14s_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df14s_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2014')

**2015**

In [ ]:
df15_simples.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva'], dtype=object)

In [ ]:
df15_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 31, 33, 35, 37, 40, 44, 46])

In [ ]:
# df15_simples.tipo_veiculo.unique()

In [ ]:
# df15_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df15_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df15_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_simples.tipo_acidente.isin(list_acid)

df15s_reta_curva_feridos = df15_simples[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15s_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df15s_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18,
       20, 21, 22, 31, 33, 35, 37, 40, 44, 46])

In [ ]:
# df15s_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df15s_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2015')

**2016**

In [ ]:
df16_simples.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df16_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 24,
       29, 34, 47, 50])

In [ ]:
# df16_simples.tipo_veiculo.unique()

In [ ]:
# df16_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df16_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df16_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_simples.tipo_acidente.isin(list_acid)

df16s_reta_curva_feridos = df16_simples[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16s_reta_curva_feridos.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
np.sort(df16s_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 18, 24, 29,
       34, 47, 50])

In [ ]:
# df16s_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df16s_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2016')

**2017**

In [ ]:
df17_simples.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel', 'Ponte',
       'Retorno Regulamentado'], dtype=object)

In [ ]:
df17_simples.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_simples.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 24, 25, 28, 31, 34, 35, 37, 45, 62])

In [ ]:
# df17_simples.tipo_veiculo.unique()

In [ ]:
# df17_simples.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df17_simples.tracado_via.isin(['Reta', 'Curva'])

feridos = df17_simples.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_simples.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_simples.tipo_acidente.isin(list_acid)

df17s_reta_curva_feridos = df17_simples[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17s_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df17s_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       24, 25, 34, 35, 37, 45, 62])

In [ ]:
# df17s_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df17s_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17s_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='simples_reta_curva_feridos_2017')

# **Estrada dupla**

In [ ]:
# 2013
prf13.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla'], dtype=object)

In [ ]:
# 2013 somente com pista "Dupla"
df13_dupla = prf13[prf13.tipo_pista.isin(['Dupla'])]
df13_dupla.tipo_pista.unique()

array(['Dupla'], dtype=object)

In [ ]:
# 2014
prf14.tipo_pista.unique()

array(['Simples', 'Múltipla', 'Dupla'], dtype=object)

In [ ]:
# 2014 somente com pista "Dupla"
df14_dupla = prf14[prf14.tipo_pista.isin(['Dupla'])]
df14_dupla.tipo_pista.unique()

array(['Dupla'], dtype=object)

In [ ]:
# 2015
prf15.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla', '(null)'], dtype=object)

In [ ]:
# 2015 somente com pista "Dupla"
df15_dupla = prf15[prf15.tipo_pista.isin(['Dupla'])]
df15_dupla.tipo_pista.unique()

array(['Dupla'], dtype=object)

In [ ]:
# 2016
prf16.tipo_pista.unique()

array(['Dupla', 'Simples', 'Múltipla'], dtype=object)

In [ ]:
# 2016 somente com pista "Dupla"
df16_dupla = prf16[prf16.tipo_pista.isin(['Dupla'])]
df16_dupla.tipo_pista.unique()

array(['Dupla'], dtype=object)

In [ ]:
# 2017
prf17.tipo_pista.unique()

array(['Simples', 'Dupla', 'Múltipla'], dtype=object)

In [ ]:
# 2017 somente com pista "Dupla"
df17_dupla = prf17[prf17.tipo_pista.isin(['Dupla'])]
df17_dupla.tipo_pista.unique()

array(['Dupla'], dtype=object)

## **Um veículo + Reta + Caminhões + Capotamento ou Tombamento**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df13_dupla.veiculos.unique()

array([1])

In [ ]:
# df13_dupla.tipo_veiculo.unique()

In [ ]:
# df13_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df13_dupla.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_dupla.tipo_acidente.isin(list_acid)

df13d_reta = df13_dupla[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13d_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df13d_reta.tipo_veiculo.unique()

In [ ]:
# df13d_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13d_reta['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_dupla.veiculos.unique()

array([1])

In [ ]:
# df14_dupla.tipo_veiculo.unique()

In [ ]:
# df14_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df14_dupla.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_dupla.tipo_acidente.isin(list_acid)

df14d_reta = df14_dupla[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14d_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df14d_reta.tipo_veiculo.unique()

In [ ]:
# df14_sd_reta_capot_tomb.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14d_reta['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df15_dupla.veiculos.unique()

array([1])

In [ ]:
# df15_dupla.tipo_veiculo.unique()

In [ ]:
# df15_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df15_dupla.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_dupla.tipo_acidente.isin(list_acid)

df15d_reta = df15_dupla[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15d_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df15d_reta.tipo_veiculo.unique()

In [ ]:
# df15d_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15d_reta['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_dupla.veiculos.unique()

array([1])

In [ ]:
# df16_dupla.tipo_veiculo.unique()

In [ ]:
# df16_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df16_dupla.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_dupla.tipo_acidente.isin(list_acid)

df16d_reta = df16_dupla[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16d_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df16d_reta.tipo_veiculo.unique()

In [ ]:
# df16d_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16d_reta['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

array(['Reta', 'Não Informado', 'Curva', 'Desvio Temporário', 'Viaduto',
       'Retorno Regulamentado', 'Rotatória', 'Ponte',
       'Interseção de vias', 'Túnel'], dtype=object)

In [ ]:
df17_dupla.veiculos.unique()

array([1])

In [ ]:
# df17_dupla.tipo_veiculo.unique()

In [ ]:
# df17_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta'  ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df17_dupla.tracado_via == 'Reta'

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_dupla.tipo_acidente.isin(list_acid)

df17d_reta = df17_dupla[(reta)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17d_reta.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
# df17d_reta.tipo_veiculo.unique()

In [ ]:
# df17d_reta.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17d_reta['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_2017')

## **Um veículo + Reta + feridos**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df13_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       21, 23, 25, 28, 32, 40, 41])

In [ ]:
# df13_dupla.tipo_veiculo.unique()

In [ ]:
# df13_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df13_dupla.tracado_via == 'Reta'

feridos = df13_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_dupla.tipo_acidente.isin(list_acid)

df13d_reta_feridos = df13_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13d_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df13d_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 14, 15, 16, 21, 28, 32, 40])

In [ ]:
# df13d_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df13d_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 16, 23, 24, 25])

In [ ]:
# df14_dupla.tipo_veiculo.unique()

In [ ]:
# df14_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df14_dupla.tracado_via == 'Reta'

feridos = df14_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_dupla.tipo_acidente.isin(list_acid)

df14d_reta_feridos = df14_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14d_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df14d_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 14, 16, 23, 25])

In [ ]:
# df14d_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df14d_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df15_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 15, 16, 31, 41])

In [ ]:
# df15_dupla.tipo_veiculo.unique()

In [ ]:
# df15_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df15_dupla.tracado_via == 'Reta'

feridos = df15_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_dupla.tipo_acidente.isin(list_acid)

df15d_reta_feridos = df15_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15d_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df15d_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 15, 31])

In [ ]:
# df15d_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df15d_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 20, 21, 23, 25, 27, 28,
       31, 41])

In [ ]:
# df16_dupla.tipo_veiculo.unique()

In [ ]:
# df16_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df16_dupla.tracado_via == 'Reta'

feridos = df16_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_dupla.tipo_acidente.isin(list_acid)

df16d_reta_feridos = df16_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16d_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df16d_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 20, 21, 23, 25, 28, 31, 41])

In [ ]:
# df16d_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df16d_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

array(['Reta', 'Não Informado', 'Curva', 'Desvio Temporário', 'Viaduto',
       'Retorno Regulamentado', 'Rotatória', 'Ponte',
       'Interseção de vias', 'Túnel'], dtype=object)

In [ ]:
df17_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 15, 16, 18, 19,
       26, 27, 28])

In [ ]:
# df17_dupla.tipo_veiculo.unique()

In [ ]:
# df17_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta = df17_dupla.tracado_via == 'Reta'

feridos = df17_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_dupla.tipo_acidente.isin(list_acid)

df17d_reta_feridos = df17_dupla[(reta) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17d_reta_feridos.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
np.sort(df17d_reta_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 15, 16, 19, 28])

In [ ]:
# df17d_reta_feridos.tipo_veiculo.unique()

In [ ]:
# df17d_reta_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17d_reta_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_feridos_2017')

## **Um veículo + Curva**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df13_dupla.veiculos.unique()

array([1])

In [ ]:
# df13_dupla.tipo_veiculo.unique()

In [ ]:
# df13_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df13_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_dupla.tipo_acidente.isin(list_acid)

df13d_curva = df13_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13d_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df13d_curva.tipo_veiculo.unique()

In [ ]:
# df13d_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13d_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_dupla.veiculos.unique()

array([1])

In [ ]:
# df14_dupla.tipo_veiculo.unique()

In [ ]:
# df14_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df14_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_dupla.tipo_acidente.isin(list_acid)

df14d_curva = df14_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14d_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df14d_curva.tipo_veiculo.unique()

In [ ]:
# df14d_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14d_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df15_dupla.veiculos.unique()

array([1])

In [ ]:
# df15_dupla.tipo_veiculo.unique()

In [ ]:
# df15_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df15_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_dupla.tipo_acidente.isin(list_acid)

df15d_curva = df15_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15d_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df15d_curva.tipo_veiculo.unique()

In [ ]:
# df15d_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15d_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_dupla.veiculos.unique()

array([1])

In [ ]:
# df16_dupla.tipo_veiculo.unique()

In [ ]:
# df16_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df16_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_dupla.tipo_acidente.isin(list_acid)

df16d_curva = df16_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16d_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df16d_curva.tipo_veiculo.unique()

In [ ]:
# df16d_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16d_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

array(['Reta', 'Não Informado', 'Curva', 'Desvio Temporário', 'Viaduto',
       'Retorno Regulamentado', 'Rotatória', 'Ponte',
       'Interseção de vias', 'Túnel'], dtype=object)

In [ ]:
df17_dupla.veiculos.unique()

array([1])

In [ ]:
# df17_dupla.tipo_veiculo.unique()

In [ ]:
# df17_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df17_dupla.tracado_via == 'Curva'

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_dupla.tipo_acidente.isin(list_acid)

df17d_curva = df17_dupla[(curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17d_curva.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
# df17d_curva.tipo_veiculo.unique()

In [ ]:
# df17d_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17d_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_2017')

## **Um veículo + Curva + feridos**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df13_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       21, 23, 25, 28, 32, 40, 41])

In [ ]:
# df13_dupla.tipo_veiculo.unique()

In [ ]:
# df13_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df13_dupla.tracado_via == 'Curva'

feridos = df13_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_dupla.tipo_acidente.isin(list_acid)

df13d_curva_feridos = df13_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13d_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df13d_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  8, 11, 12, 13, 16, 23, 25, 41])

In [ ]:
# df13d_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df13d_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 16, 23, 24, 25])

In [ ]:
# df14_dupla.tipo_veiculo.unique()

In [ ]:
# df14_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df14_dupla.tracado_via == 'Curva'

feridos = df14_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_dupla.tipo_acidente.isin(list_acid)

df14d_curva_feridos = df14_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14d_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df14d_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 11, 14, 24])

In [ ]:
# df14d_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df14d_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df15_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 15, 16, 31, 41])

In [ ]:
# df15_dupla.tipo_veiculo.unique()

In [ ]:
# df15_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df15_dupla.tracado_via == 'Curva'

feridos = df15_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_dupla.tipo_acidente.isin(list_acid)

df15d_curva_feridos = df15_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15d_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df15d_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7, 10, 11, 16, 41])

In [ ]:
# df15d_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df15d_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 20, 21, 23, 25, 27, 28,
       31, 41])

In [ ]:
# df16_dupla.tipo_veiculo.unique()

In [ ]:
# df16_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df16_dupla.tracado_via == 'Curva'

feridos = df16_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_dupla.tipo_acidente.isin(list_acid)

df16d_curva_feridos = df16_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16d_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df16d_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 27])

In [ ]:
# df16d_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df16d_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

array(['Reta', 'Não Informado', 'Curva', 'Desvio Temporário', 'Viaduto',
       'Retorno Regulamentado', 'Rotatória', 'Ponte',
       'Interseção de vias', 'Túnel'], dtype=object)

In [ ]:
df17_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 15, 16, 18, 19,
       26, 27, 28])

In [ ]:
# df17_dupla.tipo_veiculo.unique()

In [ ]:
# df17_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
curva = df17_dupla.tracado_via == 'Curva'

feridos = df17_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_dupla.tipo_acidente.isin(list_acid)

df17d_curva_feridos = df17_dupla[(curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17d_curva_feridos.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
np.sort(df17d_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 15, 27])

In [ ]:
# df17d_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df17d_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17d_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_curva_feridos_2017')

## **Um veículo + Reta + Curva**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df13_dupla.veiculos.unique()

array([1])

In [ ]:
# df13_dupla.tipo_veiculo.unique()

In [ ]:
# df13_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df13_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_dupla.tipo_acidente.isin(list_acid)

df13d_reta_curva = df13_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13d_reta_curva.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
# df13d_reta_curva.tipo_veiculo.unique()

In [ ]:
# df13d_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_dupla.veiculos.unique()

array([1])

In [ ]:
# df14_dupla.tipo_veiculo.unique()

In [ ]:
# df14_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df14_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_dupla.tipo_acidente.isin(list_acid)

df14d_reta_curva = df14_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14d_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df14d_reta_curva.tipo_veiculo.unique()

In [ ]:
# df14d_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df15_dupla.veiculos.unique()

array([1])

In [ ]:
# df15_dupla.tipo_veiculo.unique()

In [ ]:
# df15_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'      ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df15_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_dupla.tipo_acidente.isin(list_acid)

df15d_reta_curva = df15_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15d_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df15d_reta_curva.tipo_veiculo.unique()

In [ ]:
# df15d_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_dupla.veiculos.unique()

array([1])

In [ ]:
# df16_dupla.tipo_veiculo.unique()

In [ ]:
# df16_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df16_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_dupla.tipo_acidente.isin(list_acid)

df16d_reta_curva = df16_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16d_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df16d_reta_curva.tipo_veiculo.unique()

In [ ]:
# df16d_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

array(['Reta', 'Não Informado', 'Curva', 'Desvio Temporário', 'Viaduto',
       'Retorno Regulamentado', 'Rotatória', 'Ponte',
       'Interseção de vias', 'Túnel'], dtype=object)

In [ ]:
df17_dupla.veiculos.unique()

array([1])

In [ ]:
# df17_dupla.tipo_veiculo.unique()

In [ ]:
# df17_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva'     ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df17_dupla.tracado_via.isin(['Reta', 'Curva'])

# list_caminhao = ['Caminhão', 'Caminhão-trator']
# caminhao = df17_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_dupla.tipo_acidente.isin(list_acid)

df17d_reta_curva = df17_dupla[(reta_curva)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17d_reta_curva.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
# df17d_reta_curva.tipo_veiculo.unique()

In [ ]:
# df17d_reta_curva.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17d_reta_curva['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_2017')

## **Um veículo + Reta + Curva + feridos**

**2013**

In [ ]:
df13_dupla.tracado_via.unique()

array(['Curva', 'Reta', 'Cruzamento'], dtype=object)

In [ ]:
df13_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df13_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       21, 23, 25, 28, 32, 40, 41])

In [ ]:
# df13_dupla.tipo_veiculo.unique()

In [ ]:
# df13_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df13_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df13_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df13_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df13_dupla.tipo_acidente.isin(list_acid)

df13d_reta_curva_feridos = df13_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df13d_reta_curva_feridos.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
np.sort(df13d_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 21, 23,
       25, 28, 32, 40, 41])

In [ ]:
# df13d_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df13d_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df13d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2013')

**2014**

In [ ]:
df14_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df14_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df14_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 16, 23, 24, 25])

In [ ]:
# df14_dupla.tipo_veiculo.unique()

In [ ]:
# df14_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df14_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df14_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df14_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df14_dupla.tipo_acidente.isin(list_acid)

df14d_reta_curva_feridos = df14_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df14d_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df14d_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 16, 23, 24, 25])

In [ ]:
# df14d_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df14d_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df14d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2014')

**2015**

In [ ]:
df15_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df15_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df15_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 15, 16, 31, 41])

In [ ]:
# df15_dupla.tipo_veiculo.unique()

In [ ]:
# df15_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df15_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df15_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df15_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df15_dupla.tipo_acidente.isin(list_acid)

df15d_reta_curva_feridos = df15_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df15d_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df15d_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 15, 16, 31, 41])

In [ ]:
# df15d_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df15d_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df15d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2015')

**2016**

In [ ]:
df16_dupla.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
df16_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df16_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 20, 21, 23, 25, 27, 28,
       31, 41])

In [ ]:
# df16_dupla.tipo_veiculo.unique()

In [ ]:
# df16_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df16_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df16_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df16_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df16_dupla.tipo_acidente.isin(list_acid)

df16d_reta_curva_feridos = df16_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df16d_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df16d_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 20, 21, 23, 25, 27, 28, 31,
       41])

In [ ]:
# df16d_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df16d_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df16d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2016')

**2017**

In [ ]:
df17_dupla.tracado_via.unique()

array(['Reta', 'Não Informado', 'Curva', 'Desvio Temporário', 'Viaduto',
       'Retorno Regulamentado', 'Rotatória', 'Ponte',
       'Interseção de vias', 'Túnel'], dtype=object)

In [ ]:
df17_dupla.veiculos.unique()

array([1])

In [ ]:
np.sort(df17_dupla.feridos.unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 15, 16, 18, 19,
       26, 27, 28])

In [ ]:
# df17_dupla.tipo_veiculo.unique()

In [ ]:
# df17_dupla.tipo_acidente.unique()

In [ ]:
# Filtragem 'Reta' ou 'Curva' + pelo menos 1 ferido        ###Caminhões + 'Tombamento' ou 'Capotamento'
reta_curva = df17_dupla.tracado_via.isin(['Reta', 'Curva'])

feridos = df17_dupla.feridos > 0

# list_caminhao = ['Caminhão', 'Caminhão-Trator', 'Caminhão-Tanque']
# caminhao = df17_dupla.tipo_veiculo.isin(list_caminhao)

# list_acid = ['Capotamento', 'Tombamento']
# capot_tomb = df17_dupla.tipo_acidente.isin(list_acid)

df17d_reta_curva_feridos = df17_dupla[(reta_curva) & (feridos)] #& (caminhao) & (capot_tomb)]

In [ ]:
df17d_reta_curva_feridos.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
np.sort(df17d_reta_curva_feridos.feridos.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 15, 16, 19, 27, 28])

In [ ]:
# df17d_reta_curva_feridos.tipo_veiculo.unique()

In [ ]:
# df17d_reta_curva_feridos.tipo_acidente.unique()

In [ ]:
counts, bins = np.histogram(df17d_reta_curva_feridos['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road with at least one person injured (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='dupla_reta_curva_feridos_2017')

# **Reta + 1 veículo + falta de atenção + dormindo**

**2013**

In [ ]:
prf13.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
prf13.veiculos.unique()

array([1])

In [ ]:
prf13.causa_acidente.unique()

array(['Defeito mecânico em veículo', 'Velocidade incompatível', 'Outras',
       'Falta de atenção', 'Dormindo', 'Animais na Pista',
       'Ingestão de álcool', 'Ultrapassagem indevida', 'Defeito na via',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf13.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf13.causa_acidente.isin(list_causa)

df13_reta_causa = prf13[(reta) & (causa)]

In [ ]:
df13_reta_causa.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df13_reta_causa.causa_acidente.unique()

array(['Dormindo', 'Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road, sleeping or lack of attention (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2013')

**2014**

In [ ]:
prf14.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
prf14.veiculos.unique()

array([1])

In [ ]:
prf14.causa_acidente.unique()

array(['Ingestão de álcool', 'Outras', 'Falta de atenção',
       'Defeito mecânico em veículo', 'Animais na Pista', 'Dormindo',
       'Velocidade incompatível', 'Defeito na via',
       'Ultrapassagem indevida', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf14.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf14.causa_acidente.isin(list_causa)

df14_reta_causa = prf14[(reta) & (causa)]

In [ ]:
df14_reta_causa.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df14_reta_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road, sleeping or lack of attention (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2014')

**2015**

In [ ]:
prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
prf15.veiculos.unique()

array([1])

In [ ]:
prf15.causa_acidente.unique()

array(['Animais na Pista', 'Falta de atenção', 'Ingestão de álcool',
       'Outras', 'Defeito mecânico em veículo', 'Dormindo',
       'Velocidade incompatível', 'Ultrapassagem indevida',
       'Defeito na via', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf15.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf15.causa_acidente.isin(list_causa)

df15_reta_causa = prf15[(reta) & (causa)]

In [ ]:
df15_reta_causa.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df15_reta_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road, sleeping or lack of attention (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2015')

**2016**

In [ ]:
prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
prf16.veiculos.unique()

array([1])

In [ ]:
prf16.causa_acidente.unique()

array(['Falta de atenção', 'Outras', 'Dormindo', 'Ingestão de álcool',
       'Velocidade incompatível', 'Defeito mecânico em veículo',
       'Defeito na via', 'Animais na Pista', 'Ultrapassagem indevida',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
# Filtragem 'Reta' + 'Falta de atenção' ou 'Dormindo'
reta = prf16.tracado_via == 'Reta'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf16.causa_acidente.isin(list_causa)

df16_reta_causa = prf16[(reta) & (causa)]

In [ ]:
df16_reta_causa.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df16_reta_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road, sleeping or lack of attention (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2016')

**2017**

In [ ]:
prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
prf17.veiculos.unique()

array([1])

In [ ]:
prf17.causa_acidente.unique()

array(['Defeito na Via', 'Defeito Mecânico no Veículo',
       'Não guardar distância de segurança',
       'Falta de Atenção do Pedestre', 'Fenômenos da Natureza',
       'Velocidade Incompatível', 'Animais na Pista', 'Condutor Dormindo',
       'Falta de Atenção à Condução',
       'Carga excessiva e/ou mal acondicionada', 'Ultrapassagem Indevida',
       'Mal Súbito', 'Objeto estático sobre o leito carroçável',
       'Pista Escorregadia', 'Avarias e/ou desgaste excessivo no pneu',
       'Ingestão de Álcool',
       'Sinalização da via insuficiente ou inadequada',
       'Desobediência às normas de trânsito pelo condutor',
       'Restrição de Visibilidade', 'Ingestão de Substâncias Psicoativas',
       'Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo',
       'Agressão Externa',
       'Desobediência às normas de trânsito pelo pedestre'], dtype=object)

In [ ]:
# Filtragem 'Reta' + 'Falta de Atenção à Condução' ou 'Condutor Dormindo'
reta = prf17.tracado_via == 'Reta'

list_causa = ['Falta de Atenção à Condução', 'Condutor Dormindo']
causa = prf17.causa_acidente.isin(list_causa)

df17_reta_causa = prf17[(reta) & (causa)]

In [ ]:
df17_reta_causa.tracado_via.unique()

array(['Reta'], dtype=object)

In [ ]:
df17_reta_causa.causa_acidente.unique()

array(['Condutor Dormindo', 'Falta de Atenção à Condução'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_causa['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight section of the road, sleeping or lack of attention (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_dorm_atencao_2017')

# **Curva + 1 veículo + falta de atenção + dormindo**

**2013**

In [ ]:
prf13.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
prf13.veiculos.unique()

array([1])

In [ ]:
prf13.causa_acidente.unique()

array(['Defeito mecânico em veículo', 'Velocidade incompatível', 'Outras',
       'Falta de atenção', 'Dormindo', 'Animais na Pista',
       'Ingestão de álcool', 'Ultrapassagem indevida', 'Defeito na via',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf13.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf13.causa_acidente.isin(list_causa)

df13_curva_causa = prf13[(curva) & (causa)]

In [ ]:
df13_curva_causa.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df13_curva_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road, sleeping or lack of attention (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2013')

**2014**

In [ ]:
prf14.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
prf14.veiculos.unique()

array([1])

In [ ]:
prf14.causa_acidente.unique()

array(['Ingestão de álcool', 'Outras', 'Falta de atenção',
       'Defeito mecânico em veículo', 'Animais na Pista', 'Dormindo',
       'Velocidade incompatível', 'Defeito na via',
       'Ultrapassagem indevida', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf14.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf14.causa_acidente.isin(list_causa)

df14_curva_causa = prf14[(curva) & (causa)]

In [ ]:
df14_curva_causa.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df14_curva_causa.causa_acidente.unique()

array(['Dormindo', 'Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road, sleeping or lack of attention (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2014')

**2015**

In [ ]:
prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
prf15.veiculos.unique()

array([1])

In [ ]:
prf15.causa_acidente.unique()

array(['Animais na Pista', 'Falta de atenção', 'Ingestão de álcool',
       'Outras', 'Defeito mecânico em veículo', 'Dormindo',
       'Velocidade incompatível', 'Ultrapassagem indevida',
       'Defeito na via', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf15.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf15.causa_acidente.isin(list_causa)

df15_curva_causa = prf15[(curva) & (causa)]

In [ ]:
df15_curva_causa.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df15_curva_causa.causa_acidente.unique()

array(['Dormindo', 'Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road, sleeping or lack of attention (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2015')

**2016**

In [ ]:
prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
prf16.veiculos.unique()

array([1])

In [ ]:
prf16.causa_acidente.unique()

array(['Falta de atenção', 'Outras', 'Dormindo', 'Ingestão de álcool',
       'Velocidade incompatível', 'Defeito mecânico em veículo',
       'Defeito na via', 'Animais na Pista', 'Ultrapassagem indevida',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
# Filtragem 'Curva' + 'Falta de atenção' ou 'Dormindo'
curva = prf16.tracado_via == 'Curva'

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf16.causa_acidente.isin(list_causa)

df16_curva_causa = prf16[(curva) & (causa)]

In [ ]:
df16_curva_causa.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df16_curva_causa.causa_acidente.unique()

array(['Dormindo', 'Falta de atenção'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road, sleeping or lack of attention (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2016')

**2017**

In [ ]:
prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
prf17.veiculos.unique()

array([1])

In [ ]:
prf17.causa_acidente.unique()

array(['Defeito na Via', 'Defeito Mecânico no Veículo',
       'Não guardar distância de segurança',
       'Falta de Atenção do Pedestre', 'Fenômenos da Natureza',
       'Velocidade Incompatível', 'Animais na Pista', 'Condutor Dormindo',
       'Falta de Atenção à Condução',
       'Carga excessiva e/ou mal acondicionada', 'Ultrapassagem Indevida',
       'Mal Súbito', 'Objeto estático sobre o leito carroçável',
       'Pista Escorregadia', 'Avarias e/ou desgaste excessivo no pneu',
       'Ingestão de Álcool',
       'Sinalização da via insuficiente ou inadequada',
       'Desobediência às normas de trânsito pelo condutor',
       'Restrição de Visibilidade', 'Ingestão de Substâncias Psicoativas',
       'Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo',
       'Agressão Externa',
       'Desobediência às normas de trânsito pelo pedestre'], dtype=object)

In [ ]:
# Filtragem 'Curva' + 'Falta de Atenção à Condução' ou 'Condutor Dormindo'
curva = prf17.tracado_via == 'Curva'

list_causa = ['Falta de Atenção à Condução', 'Condutor Dormindo']
causa = prf17.causa_acidente.isin(list_causa)

df17_curva_causa = prf17[(curva) & (causa)]

In [ ]:
df17_curva_causa.tracado_via.unique()

array(['Curva'], dtype=object)

In [ ]:
df17_curva_causa.causa_acidente.unique()

array(['Falta de Atenção à Condução', 'Condutor Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a curve section of the road, sleeping or lack of attention (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='curva_dorm_atencao_2017')

# **Curva + Reta + 1 veículo + falta de atenção + dormindo**

**2013**

In [ ]:
prf13.tracado_via.unique()

array(['Curva', 'Cruzamento', 'Reta'], dtype=object)

In [ ]:
prf13.veiculos.unique()

array([1])

In [ ]:
prf13.causa_acidente.unique()

array(['Defeito mecânico em veículo', 'Velocidade incompatível', 'Outras',
       'Falta de atenção', 'Dormindo', 'Animais na Pista',
       'Ingestão de álcool', 'Ultrapassagem indevida', 'Defeito na via',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf13.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf13.causa_acidente.isin(list_causa)

df13_reta_curva_causa = prf13[(reta_curva) & (causa)]

In [ ]:
df13_reta_curva_causa.tracado_via.unique()

array(['Curva', 'Reta'], dtype=object)

In [ ]:
df13_reta_curva_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df13_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes13 = np.array(counts)/sum(np.array(counts))
fluxo13 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa13 = (acidentes13/fluxo13)/sum(acidentes13/fluxo13)

In [ ]:
graphics_plot_prf(y=[taxa13, fluxo13, acidentes13], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road, sleeping or lack of attention (2013) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2013')

**2014**

In [ ]:
prf14.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
prf14.veiculos.unique()

array([1])

In [ ]:
prf14.causa_acidente.unique()

array(['Ingestão de álcool', 'Outras', 'Falta de atenção',
       'Defeito mecânico em veículo', 'Animais na Pista', 'Dormindo',
       'Velocidade incompatível', 'Defeito na via',
       'Ultrapassagem indevida', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf14.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf14.causa_acidente.isin(list_causa)

df14_reta_curva_causa = prf14[(reta_curva) & (causa)]

In [ ]:
df14_reta_curva_causa.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df14_reta_curva_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df14_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes14 = np.array(counts)/sum(np.array(counts))
fluxo14 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa14 = (acidentes14/fluxo14)/sum(acidentes14/fluxo14)

In [ ]:
graphics_plot_prf(y=[taxa14, fluxo14, acidentes14], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road, sleeping or lack of attention (2014) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2014')

**2015**

In [ ]:
prf15.tracado_via.unique()

array(['Reta', 'Cruzamento', 'Curva', '(null)'], dtype=object)

In [ ]:
prf15.veiculos.unique()

array([1])

In [ ]:
prf15.causa_acidente.unique()

array(['Animais na Pista', 'Falta de atenção', 'Ingestão de álcool',
       'Outras', 'Defeito mecânico em veículo', 'Dormindo',
       'Velocidade incompatível', 'Ultrapassagem indevida',
       'Defeito na via', 'Não guardar distância de segurança',
       'Desobediência à sinalização'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf15.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf15.causa_acidente.isin(list_causa)

df15_reta_curva_causa = prf15[(reta_curva) & (causa)]

In [ ]:
df15_reta_curva_causa.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df15_reta_curva_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df15_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes15 = np.array(counts)/sum(np.array(counts))
fluxo15 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa15 = (acidentes15/fluxo15)/sum(acidentes15/fluxo15)

In [ ]:
graphics_plot_prf(y=[taxa15, fluxo15, acidentes15], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road, sleeping or lack of attention (2015) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2015')

**2016**

In [ ]:
prf16.tracado_via.unique()

array(['Reta', 'Curva', 'Cruzamento'], dtype=object)

In [ ]:
prf16.veiculos.unique()

array([1])

In [ ]:
prf16.causa_acidente.unique()

array(['Falta de atenção', 'Outras', 'Dormindo', 'Ingestão de álcool',
       'Velocidade incompatível', 'Defeito mecânico em veículo',
       'Defeito na via', 'Animais na Pista', 'Ultrapassagem indevida',
       'Desobediência à sinalização',
       'Não guardar distância de segurança'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de atenção' ou 'Dormindo'
reta_curva = prf16.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de atenção', 'Dormindo']
causa = prf16.causa_acidente.isin(list_causa)

df16_reta_curva_causa = prf16[(reta_curva) & (causa)]

In [ ]:
df16_reta_curva_causa.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df16_reta_curva_causa.causa_acidente.unique()

array(['Falta de atenção', 'Dormindo'], dtype=object)

In [ ]:
counts, bins = np.histogram(df16_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes16 = np.array(counts)/sum(np.array(counts))
fluxo16 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa16 = (acidentes16/fluxo16)/sum(acidentes16/fluxo16)

In [ ]:
graphics_plot_prf(y=[taxa16, fluxo16, acidentes16], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road, sleeping or lack of attention (2016) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2016')

**2017**

In [ ]:
prf17.tracado_via.unique()

array(['Desvio Temporário', 'Não Informado', 'Reta', 'Curva', 'Viaduto',
       'Rotatória', 'Interseção de vias', 'Túnel',
       'Retorno Regulamentado', 'Ponte'], dtype=object)

In [ ]:
prf17.veiculos.unique()

array([1])

In [ ]:
prf17.causa_acidente.unique()

array(['Defeito na Via', 'Defeito Mecânico no Veículo',
       'Não guardar distância de segurança',
       'Falta de Atenção do Pedestre', 'Fenômenos da Natureza',
       'Velocidade Incompatível', 'Animais na Pista', 'Condutor Dormindo',
       'Falta de Atenção à Condução',
       'Carga excessiva e/ou mal acondicionada', 'Ultrapassagem Indevida',
       'Mal Súbito', 'Objeto estático sobre o leito carroçável',
       'Pista Escorregadia', 'Avarias e/ou desgaste excessivo no pneu',
       'Ingestão de Álcool',
       'Sinalização da via insuficiente ou inadequada',
       'Desobediência às normas de trânsito pelo condutor',
       'Restrição de Visibilidade', 'Ingestão de Substâncias Psicoativas',
       'Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo',
       'Agressão Externa',
       'Desobediência às normas de trânsito pelo pedestre'], dtype=object)

In [ ]:
# Filtragem 'Reta' ou 'Curva' + 'Falta de Atenção à Condução' ou 'Condutor Dormindo'
reta_curva = prf17.tracado_via.isin(['Reta', 'Curva'])

list_causa = ['Falta de Atenção à Condução', 'Condutor Dormindo']
causa = prf17.causa_acidente.isin(list_causa)

df17_reta_curva_causa = prf17[(reta_curva) & (causa)]

In [ ]:
df17_reta_curva_causa.tracado_via.unique()

array(['Reta', 'Curva'], dtype=object)

In [ ]:
df17_reta_curva_causa.causa_acidente.unique()

array(['Condutor Dormindo', 'Falta de Atenção à Condução'], dtype=object)

In [ ]:
counts, bins = np.histogram(df17_reta_curva_causa['Hora'], bins=range(0, 25, 1))
acidentes17 = np.array(counts)/sum(np.array(counts))
fluxo17 = np.array(dfDNIT[0]/sum(dfDNIT[0]))

taxa17 = (acidentes17/fluxo17)/sum(acidentes17/fluxo17)

In [ ]:
graphics_plot_prf(y=[taxa17, fluxo17, acidentes17], 
                  y_name=['Accidents/Traffic', 'Traffic','Accidents (raw)'],
                  bins=24, yaxes_title=['Accidents/Traffic, Traffic','Accidents'], 
                  title='One vehicle on a straight or curve section of the road, sleeping or lack of attention (2017) - Accidents, traffic and accidents / traffic',
                  x_tick=5, save=True, save_name='reta_curva_dorm_atencao_2017')

# **Análise estatística**

In [ ]:
ACCE3, binsE3 = np.histogram(dfOCO_TM_E3['HorarioAcidente'], bins=range(0, 25, 1))
ACCE5, binsE5 = np.histogram(dfOCO_TM_E5['HorarioAcidente'], bins=range(0, 25, 1))
ACCE9, binsE9 = np.histogram(dfOCO_TM_E9['HorarioAcidente'], bins=range(0, 25, 1))

In [ ]:
# Define o deslocamento e agrupamento
des = 0
agr = 2
Nbins = int((24/agr)+1)

In [ ]:
# Motoristas para cada compania
m3 = np.array(dfE3_final['motoristasE3'])
m3 = np.roll(m3, des)
m3r = m3.reshape((int(24/agr),agr)).sum(axis=1)

m5 = np.array(dfE5_final['motoristasE5'])
m5 = np.roll(m5, des)
m5r = m5.reshape((int(24/agr),agr)).sum(axis=1)

m9 = np.array(dfE9_final['motoristasE9'])
m9 = np.roll(m9, des)
m9r = m9.reshape((int(24/agr),agr)).sum(axis=1)

In [ ]:
# Acidentes para cada compania
a3 = np.array(ACCE3)
a3 = np.roll(a3, des)
a3r = a3.reshape((int(24/agr),agr)).sum(axis=1)

a5 = np.array(ACCE5)
a5 = np.roll(a5, des)
a5r = a5.reshape((int(24/agr),agr)).sum(axis=1)

a9 = np.array(ACCE9)
a9 = np.roll(a9, des)
a9r = a9.reshape((int(24/agr),agr)).sum(axis=1)

In [ ]:
## Variáveis da função graphics_plot
Nbins = 25
ticks = 5

acc = a3+a5+a9
drivers = m3+m5+m9

graphics_plot(y = [acc, acc/drivers], y_name = ['Events(raw)', 'Events/Drivers'], 
              bins = Nbins, title = 'Total accidents', yaxes_title = ['Raw', 'Rate'], 
              x_tick = 5, save=True)

In [ ]:
# sensores
of3, bins = np.histogram(dfE3['HORA'], bins=range(0, 25, 1))

of3 = np.roll(of3, des)
of3r = of3.reshape((int(24/agr),agr)).sum(axis=1)

of5, bins = np.histogram(dfE5_of['HORA'], bins=range(0, 25, 1))

of5 = np.roll(of5, des)
of5r = of5.reshape((int(24/agr),agr)).sum(axis=1)

y5, bins = np.histogram(dfE5_b['HORA'], bins=range(0, 25, 1))

y5 = np.roll(y5, des)
y5r = y5.reshape((int(24/agr),agr)).sum(axis=1)

y9, bins = np.histogram(dfE9_yawn['HORA'], bins=range(0, 25, 1))

y9 = np.roll(y9, des)
y9r = y9.reshape((int(24/agr),agr)).sum(axis=1)

of9, bins = np.histogram(dfE9_tired['HORA'], bins=range(0, 25, 1))

of9 = np.roll(of9, des)
of9r = of9.reshape((int(24/agr),agr)).sum(axis=1)

at9, bins = np.histogram(dfE92H['HORA'], bins=range(0, 25, 1))

at9 = np.roll(at9, des)
at9r = at9.reshape((int(24/agr),agr)).sum(axis=1)

In [ ]:
# Referência
b = np.roll(taxa, des)
cfedr = b.reshape((int(24/agr),agr)).sum(axis=1)

In [ ]:
graphics_plot(y = [stats.zscore(of3r/m3r), stats.zscore(of5r/m5r), 
                   stats.zscore(of9r/m9r), stats.zscore(cfedr)], 
              y_name = ['E3','E5','E9','Accidents rate'], bins = Nbins, 
              title = 'Comparison companies and Brazilian Highways (Closed Eyes)', 
              yaxes_title = ['Closed eyes rate','Federal Highways'], x_tick = 5,
              save=True)

In [ ]:
graphics_plot(y = [stats.zscore(a3r/m3r), stats.zscore(a5r/m5r), 
                   stats.zscore(a9r/m9r), stats.zscore(cfedr)], 
              y_name = ['E3','E5','E9','Accidents rate'], bins = Nbins, 
              title = 'Comparison companies and Brazilian Highways (Accidents)', 
              yaxes_title = ['Accidents rate','Federal Highways'], x_tick = 5,
              save=True)

In [ ]:
graphics_plot(y = [stats.zscore(y5r/m5r), stats.zscore(y9r/m9r), stats.zscore(cfedr)], 
              y_name = ['E5','E9','Accidents rate'], bins = Nbins, 
              title = 'Comparison companies and Brazilian Highways (Yawn)', 
              yaxes_title = ['Yawn rate','Federal Highways'], x_tick = 5,
              save=True)

In [ ]:
graphics_plot(y = [stats.zscore(at9r/m9r), stats.zscore(cfedr)], 
              y_name = ['E9','Accidents Rate'], bins = Nbins, 
              title = 'Comparison companies and Brazilian Highways (Attention Warning)', 
              yaxes_title = ['Attention warning rate','Federal Highways'], x_tick = 5,
              save=True)

## **Ajustando uma senóide**

In [ ]:
def sinusoid(x, A, offset, omega, phase):
 return A*np.sin(2*np.pi*x/24+phase) + offset

In [ ]:
T = 24
def get_p0(x, y): 
 A0 = (max(y[0:T]) - min(y[0:T]))/2
 offset0 = y[0]
 phase0 = 0
 omega0 = 2.*np.pi/T
 return [A0, offset0, omega0, phase0]

In [ ]:
Nbins = 13
X = np.linspace(0, 24, Nbins)
X = X[0:12]
ys = cfedr
acs = at9r/m9r

In [ ]:
paramy, covariancey = curve_fit(sinusoid, X, ys, p0=get_p0(X, ys))
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X, acs))

In [ ]:
graphics_plot(y=[stats.zscore(sinusoid(X, *paramy)), stats.zscore(ys), stats.zscore(acs), 
                 stats.zscore(sinusoid(X, *parama))], 
              y_name=['Selected accidents - fit', 'Selected accidents - raw',
                      'Federal Highways - raw', 'Federal Highways - fit'],
              yaxes_title=['Fit'], title='Fitting an harmonic curve to the accidents',
              bins=Nbins, x_tick=5)

In [ ]:
#1
graphics_plot(y=[stats.zscore(ys), stats.zscore(sinusoid(X, *paramy))], 
              y_name=['Selected accidents - fit', 'Selected accidents - raw'],
              yaxes_title=['Z-Score'], 
              title='Fitting an harmonic curve to the accidents (Federal Highways)',
              bins=Nbins, x_tick=5, save=True)

In [ ]:
Y = stats.zscore(ys)
X_m = stats.zscore(sinusoid(X, *paramy))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y,X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]

In [ ]:
acs = (a3r+a5r+a9r)/(m3r+m5r+m9r)

In [ ]:
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X,acs))

In [ ]:
#2 Acidentes das 3 empresas
graphics_plot(y = [stats.zscore(acs), stats.zscore(sinusoid(X, *parama))], 
              y_name=['Selected accidents - raw', 'Selected accidents - fit'], 
              yaxes_title=['Z-Score'], bins=Nbins, x_tick=5, save=True,
              title='Fitting an harmonic curve to the accidents (Total accidents)')

In [ ]:
Y = stats.zscore(acs)
X_m = stats.zscore(sinusoid(X, *parama))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y, X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]

In [ ]:
acs = y9r/m9r

In [ ]:
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X,acs))

In [ ]:
#3 Yawns E9
graphics_plot(y = [stats.zscore(acs), stats.zscore(sinusoid(X, *parama))], 
              y_name=['Selected accidents - raw', 'Selected accidents - fit'],
              yaxes_title=['Z-Score'], bins=Nbins, x_tick=5, save=True,
              title='Fitting an harmonic curve to the accidents (Yawns E9)')

In [ ]:
Y = stats.zscore(acs)
X_m = stats.zscore(sinusoid(X, *parama))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y,X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]

In [ ]:
acs = at9r/m9r

In [ ]:
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X,acs))

In [ ]:
#4 Aten Warning E9
graphics_plot(y = [stats.zscore(acs), stats.zscore(sinusoid(X, *parama))], 
              y_name=['Selected accidents - raw', 'Selected accidents - fit'], 
              yaxes_title=['Z-Score'], bins=Nbins, x_tick=5, save=True,
              title='Fitting an harmonic curve to the accidents (Atten-Waring E9)')

In [ ]:
Y = stats.zscore(acs)
X_m = stats.zscore(sinusoid(X, *parama))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y,X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]

In [ ]:
acs = of9r/m9r

In [ ]:
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X, acs))

In [ ]:
#5 Eye closed E9
graphics_plot(y = [stats.zscore(acs), stats.zscore(sinusoid(X, *parama))], 
              y_name=['Selected accidents - raw', 'Selected accidents - fit'], 
              yaxes_title=['Z-Score'], bins=Nbins, x_tick=5, save=True,
              title='Fitting an harmonic curve to the accidents (Eye closed E9)')

In [ ]:
Y = stats.zscore(acs)
X_m = stats.zscore(sinusoid(X, *parama))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y,X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]

In [ ]:
acs = of3r/m3r

In [ ]:
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X,acs))

In [ ]:
#6 Eye closed E3
graphics_plot(y = [stats.zscore(acs), stats.zscore(sinusoid(X, *parama))], 
              y_name=['Selected accidents - raw', 'Selected accidents - fit'], 
              yaxes_title=['Z-Score'], bins=Nbins, x_tick=5, save=True,
              title='Fitting an harmonic curve to the accidents (Eye closed E3)')

In [ ]:
Y = stats.zscore(acs)
X_m = stats.zscore(sinusoid(X, *parama))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y,X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]

In [ ]:
acs = of5r/m5r

In [ ]:
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X,acs))

In [ ]:
#7 Eye closed E5
graphics_plot(y = [stats.zscore(acs), -stats.zscore(sinusoid(X, *paramy))], 
              y_name=['Selected accidents - raw', 'Selected accidents - fit'], 
              yaxes_title=['Z-Score'], bins=Nbins, x_tick=5, save=True,
              title='Fitting an harmonic curve to the accidents (Eye closed E5)')

In [ ]:
Y = stats.zscore(acs)
X_m = stats.zscore(sinusoid(X, *parama))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y,X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]

In [ ]:
acs = y5r/m5r

In [ ]:
parama, covariancea = curve_fit(sinusoid, X, acs, p0=get_p0(X,acs))

In [ ]:
#8 Yawns E5
graphics_plot(y = [stats.zscore(acs), -stats.zscore(sinusoid(X, *paramy))], 
              y_name=['Selected accidents - raw', 'Selected accidents - fit'], 
              yaxes_title=['Z-Score'], bins=Nbins, x_tick=5, save=True,
              title='Fitting an harmonic curve to the accidents (Yawns E5)')

In [ ]:
Y = stats.zscore(acs)
X_m = stats.zscore(sinusoid(X, *parama))
X_m = sm.add_constant(X_m)
model = sm.OLS(Y,X_m)
results = model.fit()
results.params

y5 = results.predict(X_m)
print(results.summary())

In [ ]:
results.pvalues[1]